In [54]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import math

from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
from common.utils import load_data, mape
from IPython.display import Image

%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format
np.set_printoptions(precision=2)
warnings.filterwarnings("ignore") # specify to ignore warning messages

In [55]:
# Load data
data = './data/darwin.txt'
df = pd.read_csv(data, delim_whitespace=True, skiprows=2)
df = df.dropna()
df.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
0,1951,5.30,5.10,7.00,9.40,11.90,11.2,13.30,13.20,12.50,11.40,9.50,8.90
1,1952,6.70,7.10,8.30,10.30,10.30,12.5,13.00,12.60,12.10,9.90,7.70,8.40
2,1953,6.90,7.30,8.00,8.90,13.10,12.5,13.90,13.00,13.20,10.80,9.40,8.10
3,1954,4.90,7.30,7.70,7.70,10.10,12.3,12.40,11.80,11.90,10.70,9.10,6.50
4,1955,8.30,3.70,6.70,8.90,9.40,10.9,11.70,11.40,10.90,9.30,7.60,7.60


In [64]:
tmpdf = df.drop(columns="YEAR")
tmpdf = tmpdf.dropna()
# df2 = tmpdf.sum(axis=0)



In [77]:
df2 = df.T
df2

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
YEAR,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
JAN,5.30,6.70,6.90,4.90,8.30,4.80,6.50,8.50,6.10,6.50,...,6.00,5.80,5.00,6.50,10.20,6.00,4.80,6.80,7.20,5.60
FEB,5.10,7.10,7.30,7.30,3.70,4.70,6.40,7.10,8.70,6.70,...,7.20,7.20,6.00,7.80,8.60,7.10,7.00,9.00,7.40,5.00
MAR,7.00,8.30,8.00,7.70,6.70,6.90,7.80,8.10,6.90,7.10,...,6.30,7.20,9.10,8.90,9.20,7.40,6.10,8.40,9.10,8.10
APR,9.40,10.30,8.90,7.70,8.90,8.30,9.50,8.90,8.90,8.50,...,10.70,9.70,8.90,10.50,10.80,9.70,9.50,10.10,10.50,9.70
MAY,11.90,10.30,13.10,10.10,9.40,9.10,12.30,10.90,10.40,10.00,...,11.30,10.80,11.70,13.10,10.90,11.70,12.20,12.30,12.40,11.30
JUN,11.2,12.5,12.5,12.3,10.9,11.3,12.5,11.6,12.8,12.5,...,13.3,11.2,12.5,13.3,12.2,13.5,13.7,13.8,12.9,13.1
JUL,13.30,13.00,13.90,12.40,11.70,11.20,12.90,12.40,13.50,12.50,...,12.80,12.50,13.90,15.00,13.10,13.20,12.80,14.20,12.70,12.30
AUG,13.20,12.60,13.00,11.80,11.40,11.50,13.30,11.60,13.10,12.50,...,14.00,13.10,14.50,14.80,12.40,13.00,13.10,14.30,12.50,13.30
SEP,12.50,12.10,13.20,11.90,10.90,11.40,13.10,11.50,11.90,10.50,...,12.20,11.60,13.10,13.90,11.00,12.30,12.30,14.50,11.60,12.00


In [76]:
df2

df_times = pd.DataFrame()
df_times['value'] = 0

for column in df2.columns:
    df_times.join(df2[column].iloc[1:], on='value')

print(df2[0].iloc[1:])

JAN    5.30
FEB    5.10
MAR    7.00
APR    9.40
MAY   11.90
JUN    11.2
JUL   13.30
AUG   13.20
SEP   12.50
OCT   11.40
NOV    9.50
DEC    8.90
Name: 0, dtype: object


,value
